In [1]:
import pandas as pd
jurnal = pd.read_csv(r"C:\Users\Fauzan Anshori\Downloads\Jurnal ML\datajurnal2.csv",sep=";")

In [2]:
import nltk
import nltk.tokenize


# Tokenizing
nltk.download('punkt')
# Stopwords removal
nltk.download('stopwords')
# Lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

print('Completed')

Completed


[nltk_data] Downloading package punkt to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
import re

def clean_title(title):
  re.sub("[^a-zA-Z0-9 ]", "", title)
  return title

def clean_abstract(abstract):
  re.sub("[a-zA-Z ]", "", abstract)
  return abstract

jurnal["clean_title"] = jurnal["title"].apply(clean_title)
jurnal["clean_abstract"] = jurnal["abstract"].apply(clean_abstract)

In [4]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(jurnal["clean_abstract"])

cosine_sim_matrix = cosine_similarity(tfidf_matrix)

def get_recommendations(cosine_sim_matrix, titles, user_input, top_n=5):
    user_tfidf = vectorizer.transform([user_input])
    user_cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix)
    
    sim_scores = list(enumerate(user_cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Include the first one (itself) as well
    sim_scores = sim_scores[:top_n]
    
    return [titles[i[0]] for i in sim_scores]

# Wrap everything into a single dictionary
model_data = {
    'vectorizer': vectorizer,
    'cosine_similarity': cosine_sim_matrix,
    'documents': jurnal["clean_abstract"],
    'titles': jurnal["clean_title"],
    'get_recommendations': get_recommendations
}

# Save the model data as a pickle file
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("Model saved as 'recommendation_model.pkl'")

Model saved as 'recommendation_model.pkl'


In [ ]:
# Load the saved model from the pickle file
with open('recommendation_model.pkl', 'rb') as f:
    model_data = pickle.load(f)

# Extract the components
loaded_vectorizer = model_data['vectorizer']
loaded_documents = model_data['documents']
loaded_titles = model_data['titles']
loaded_cosine_sim = model_data['cosine_similarity']
get_recommendations = model_data['get_recommendations']

# Take user input
user_input = input("Enter the journal text you are interested in: ")

# Get recommendations based on the user's input
recommendations = get_recommendations(loaded_cosine_sim, loaded_titles, user_input)

# Display the recommendations
print("Recommended Journals/Articles:")
for rec in recommendations:
    print(rec)
